## Call api get data moodle

In [12]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML

# Cấu hình Moodle API
moodle_url = "http://localhost:8100/webservice/rest/server.php"
api_token = "884fbaababf6111cc6f192021c9309b6"

def call_moodle_api(wsfunction, params=None):
    """
    Gọi Moodle Web Service API với hàm và tham số được chỉ định.

    Args:
        wsfunction (str): Tên hàm Web Service (e.g., 'core_course_get_courses').
        params (dict, optional): Tham số bổ sung cho API.

    Returns:
        dict: Dữ liệu JSON trả về từ API hoặc None nếu có lỗi.
    """
    # Tạo tham số mặc định
    default_params = {
        'wstoken': api_token,
        'wsfunction': wsfunction,
        'moodlewsrestformat': 'json'
    }
    
    # Gộp tham số bổ sung (nếu có)
    if params:
        default_params.update(params)
    
    try:
        # Gửi yêu cầu GET tới Moodle API
        response = requests.get(moodle_url, params=default_params)
        
        # Kiểm tra mã trạng thái HTTP
        if response.status_code != 200:
            print(f"❌ Lỗi HTTP: {response.status_code}")
            print("Nội dung phản hồi:", response.text)
            return None
        
        # Chuyển đổi phản hồi thành JSON
        data = response.json()
        
        # In phản hồi JSON được định dạng đẹp
        print(f"Phản hồi từ '{wsfunction}':")
        print(json.dumps(data, indent=2, ensure_ascii=False))
        
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gửi yêu cầu: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Lỗi phân tích JSON: {e}")
        print("Nội dung phản hồi:", response.text)
        return None

def get_course_module(cmid):
    """
    Lấy thông tin chi tiết của một module khóa học theo cmid.

    Args:
        cmid (int): ID của course module.

    Returns:
        dict: Thông tin module hoặc None nếu có lỗi.
    """
    return call_moodle_api('core_course_get_course_module', {'cmid': cmid})

def get_all_courses():
    """
    Lấy danh sách tất cả các khóa học trong Moodle.

    Returns:
        list: Danh sách các khóa học hoặc None nếu có lỗi.
    """
    return call_moodle_api('core_course_get_courses')

def get_site_info():
    """
    Lấy thông tin cấu hình của trang Moodle.

    Returns:
        dict: Thông tin trang hoặc None nếu có lỗi.
    """
    return call_moodle_api('core_webservice_get_site_info')

def get_course_contents(courseid):
    """
    Lấy chi tiết nội dung của một khóa học, bao gồm các section và module.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        list: Danh sách các section hoặc None nếu có lỗi.
    """
    data = call_moodle_api('core_course_get_contents', {'courseid': courseid})
    if data:
        # Hiển thị thông tin section và module dưới dạng bảng
        display_course_contents(data)
    return data

def display_course_contents(sections):
    """
    Hiển thị thông tin các section và module trong khóa học dưới dạng bảng.

    Args:
        sections (list): Danh sách các section từ API core_course_get_contents.
    """
    for section in sections:
        print(f"\n📚 Section: {section['name']} (ID: {section['id']})")
        print(f"Tóm tắt: {section.get('summary', 'Không có tóm tắt')}")
        
        if section['modules']:
            # Tạo danh sách module để hiển thị trong bảng
            module_data = [
                {
                    'Module ID': module['id'],
                    'Tên Module': module['name'],
                    'Loại Module': module['modname'],
                    'URL': module.get('url', 'Không có URL'),
                    'Mô tả': module.get('description', 'Không có mô tả')
                }
                for module in section['modules']
            ]
            df = pd.DataFrame(module_data)
            display(HTML(df.to_html(index=False)))
        else:
            print("Không có module trong section này.")

def get_lesson_details(lessonid):
    """
    Lấy chi tiết một bài học (lesson) theo lessonid.

    Args:
        lessonid (int): ID của bài học.

    Returns:
        dict: Thông tin bài học hoặc None nếu có lỗi.
    """
    data = call_moodle_api('mod_lesson_get_lesson', {'lessonid': lessonid})
    if data:
        # Hiển thị thông tin bài học
        print(f"\n📚 Bài học: {data.get('name', 'Không có tên')} (ID: {lessonid})")
        print(f"Mô tả: {data.get('intro', 'Không có mô tả')}")
    return data

def get_lesson_pages(lessonid):
    """
    Lấy danh sách các trang (pages) trong một bài học.

    Args:
        lessonid (int): ID của bài học.

    Returns:
        list: Danh sách các trang hoặc None nếu có lỗi.
    """
    data = call_moodle_api('mod_lesson_get_pages', {'lessonid': lessonid})
    if data and isinstance(data, list):
        display_lesson_pages(data, lessonid)
    return data

def display_lesson_pages(pages, lessonid):
    """
    Hiển thị danh sách các trang trong bài học dưới dạng bảng.

    Args:
        pages (list): Danh sách các trang từ API mod_lesson_get_pages.
        lessonid (int): ID của bài học.
    """
    if not pages:
        print(f"Không có trang nào trong bài học {lessonid}.")
        return
    
    try:
        page_data = [
            {
                'Page ID': page['page']['id'],
                'Tiêu đề': page['page']['title'],
                'Loại': page['page']['type'],
                'Nội dung': page['page'].get('contents', 'Không có nội dung')
            }
            for page in pages
        ]
        df = pd.DataFrame(page_data)
        print(f"\n📖 Các trang trong bài học (lessonid={lessonid}):")
        display(HTML(df.to_html(index=False)))
    except (KeyError, TypeError) as e:
        print(f"❌ Lỗi khi xử lý dữ liệu trang: {e}")
        print("Dữ liệu nhận được:", json.dumps(pages, indent=2, ensure_ascii=False))

def get_course_contents(courseid):
    """
    Lấy chi tiết nội dung của một khóa học để tìm lessonid.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        list: Danh sách các section hoặc None nếu có lỗi.
    """
    data = call_moodle_api('core_course_get_contents', {'courseid': courseid})
    if data:
        # Tìm lessonid từ các module
        lesson_ids = []
        for section in data:
            for module in section['modules']:
                if module['modname'] == 'lesson':
                    lesson_ids.append(module['instance'])
        if lesson_ids:
            print(f"Các lessonid tìm thấy trong khóa học {courseid}: {lesson_ids}")
        else:
            print(f"Không tìm thấy lesson nào trong khóa học {courseid}.")
        return lesson_ids
    return []

def get_log_module(cmid):
    """
    Lấy danh sách các trang (pages) trong một bài học.

    Args:
        cmid (int): ID của module khóa học.

    Returns:
        list: Danh sách các trang hoặc None nếu có lỗi.
    """
    data = call_moodle_api('local_userlog_get_module_logs', {'cmid': cmid})
    # if data and isinstance(data, list):
        # display_lesson_pages(data, cmid)
    return data

# Gọi các hàm để kiểm tra
if __name__ == "__main__":
    # print("=== Lấy thông tin trang Moodle ===")
    # get_site_info()
    
    # print("\n=== Lấy danh sách tất cả khóa học ===")
    # get_all_courses()
    
    # print("\n=== Lấy thông tin module khóa học (cmid=16) ===")
    # get_course_module(16)
    
    # print("=== Lấy chi tiết khóa học (courseid=3) ===")
    # get_course_contents(3)
    
    # get_lesson_details(3)  # Thay đổi ID bài học nếu cần
    # get_lesson_pages(4)  # Thay đổi ID bài học nếu cần
    print("\n=== Lấy log module (cmid=17) ===")
    data = get_log_module(17)
    print(json.dumps(data, indent=2, ensure_ascii=False))


=== Lấy log module (cmid=17) ===
Phản hồi từ 'local_userlog_get_module_logs':
[
  {
    "id": 2155,
    "userid": 2,
    "username": "Nguyen Loc",
    "eventname": "\\mod_lesson\\event\\content_page_viewed",
    "action": "viewed",
    "target": "content_page",
    "component": "mod_lesson",
    "timecreated": 1752647210
  },
  {
    "id": 2154,
    "userid": 2,
    "username": "Nguyen Loc",
    "eventname": "\\mod_lesson\\event\\course_module_viewed",
    "action": "viewed",
    "target": "course_module",
    "component": "mod_lesson",
    "timecreated": 1752647210
  },
  {
    "id": 2107,
    "userid": 2,
    "username": "Nguyen Loc",
    "eventname": "\\mod_lesson\\event\\content_page_viewed",
    "action": "viewed",
    "target": "content_page",
    "component": "mod_lesson",
    "timecreated": 1752646406
  },
  {
    "id": 2106,
    "userid": 2,
    "username": "Nguyen Loc",
    "eventname": "\\mod_lesson\\event\\course_module_viewed",
    "action": "viewed",
    "target": "cours